# Load tapes and lob data

do this for each day otherwise memory (RAM) exceeds most computers

In [61]:
# code
from fast_tools import get_data, get_data_gen

#data = get_data()

# Clean Data
Remove outliers from lob and create an additional columns noting this

FFill tapes data to get the most up to date tapes price

In [62]:
# code
import numpy as np
from numba import njit, prange

def get_tapes_window(tapes):
    dt = 60*60 # in seconds
    #stds = []
    #means = []
    w_bids = []
    w_asks = []

    t_start = 0
    start_time = 0
    z = 3.29 # 99.9%

    outside = []
    while True:
        end_time = start_time + dt
        t_end = t_start
        rolling_tapes = []
        while tapes[t_end, 0] < end_time:
            rolling_tapes += [tapes[t_end, 1]] * int(tapes[t_end,2])
            t_end += 1

        mean = np.mean(rolling_tapes)
        std = np.std(rolling_tapes)
        #means.append(mean)

        w_bid = mean - std * z
        w_ask = mean + std * z
        w_bids.append(w_bid)
        w_asks.append(w_ask)

        # look one minute a head
        local_end = t_end
        future_tapes = []
        while tapes[local_end, 0] < end_time + 60:
            future_tapes += [tapes[local_end, 1]] * int(tapes[local_end,2])
            local_end += 1

        future_tapes = np.array(future_tapes)

        n_above = len(np.where(future_tapes > w_ask)[0])
        n_below = len(np.where(future_tapes < w_bid)[0])
        if end_time % 60 !=0:
            raise ValueError
        outside.append((end_time,n_above, n_below, len(future_tapes)))

        start_time += 60
        while tapes[t_start, 0] < start_time:
            t_start += 1

        end_time += dt
        if end_time >= 8.5*60*60:
            break

    return outside, w_bids, w_asks

@njit(parallel=True)
def get_features(lob_data: np.array, 
                 lob_times: np.array, 
                 tapes: np.array, 
                 time_step_s: int, 
                 window_data: np.array,
                 ab_weight = 1, 
                 median = False, 
                 ):
    """
    Calculate features from LOB and Tapes data.

    Parameters:
    -----------
    lob_data : np.array
        Array containing the limit order book (LOB) data.
    lob_times : np.array
        Array containing timestamps for the LOB data.
    tapes : np.array
        Array containing Tapes data.
    time_step_s : int
        Time step in seconds for calculating features.
    ab_weight : float, optional
        Weight parameter for alpha and beta calculations, by default 1.
    median : bool, optional
        Whether to calculate features using median instead of mean, by default False.
    window_data : np.array
        Array containing window data for calculating CBS and CAS, by default None.

    Returns:
    --------
    tuple
        A tuple containing:
        - feat_arr: np.array
            Array containing feature values.
        - features: list
            List of feature names.
    """
    
    n_rows = int((8.5 * 60 * 60) / time_step_s)                         # define number of rows of output array
    features = ["MP","HIBID","LOASK","AP","WBP","WAP",                  # define features
                "TCBS","TCAS","AWS","VOL","GAP","SPREAD",
                "ALPHA", "BETA", "ZETA", "ENDT", 
                "PSTD", "LOWIN", "HIWIN", "nUoD"]
    n_features = len(features)                                          # define number of features

    feat_arr = np.zeros((n_rows, n_features), dtype=np.float64)         # array to hold feature values
    
    LA_HB_a_b = np.zeros((lob_data.shape[0]+1, 4), dtype = np.float64)  # array holding the LOASK, HIBID,
                                                                        # alpha, beta, values 

    for i in prange(lob_data.shape[0]):                                 # iterates over the LOB to fill
        row = lob_data[i]                                               # LA_HB_a_b values
        
        neg_ind = np.where(row < 0)[0]                                  # locate bid and ask prices (indicies)
        pos_ind = np.where(row > 0)[0]
        
        if len(neg_ind) == 0:                                           # assign HIBID, np.nan if no values
            LA_HB_a_b[i][1] = np.nan
        else:
            LA_HB_a_b[i][1] = max(neg_ind) + 1 

        if len(pos_ind) == 0:                                           # assign HIBID, np.nan if no values
            LA_HB_a_b[i][0] = np.nan
        else:
            LA_HB_a_b[i][0] = min(pos_ind) + 1

        mid_price = (LA_HB_a_b[i][0] + LA_HB_a_b[i][1]) / 2             # calculate mid_price for alpha/beta calculations

        if np.isnan(mid_price):
            alpha = np.nan
            beta = np.nan
        else:                                                           # calculate alpha/beta using ab_weight var
            beta = 0
            for ind in neg_ind:
                beta += (-1 * row[ind]) / ((mid_price - (ind + 1)) + ab_weight)
    
            alpha = 0
            for ind in pos_ind:
                alpha += row[ind] / (((ind + 1) - mid_price) + ab_weight)
                

        LA_HB_a_b[i][2] = alpha
        LA_HB_a_b[i][3] = beta
        
    max_lob = lob_data.shape[0] - 1                                      # define max indicies for lob
    max_tapes = tapes.shape[0] - 1                                       # define max indicies for tapes
    
    start_time = 0                                                       # define start time
    lob_start = 0                                                        # define start index for lob
    tapes_start = 0                                                      # define start index for tapes
    
    cas = np.zeros(800, dtype = np.int16)                                # define an array to hold CAS values
    cbs = np.zeros(800, dtype = np.int16)                                # define an array to hold CBS values
    for row_i in range(n_rows):
        end_time = start_time + time_step_s                              # move to next time step
        lob_end = lob_start
        tapes_end = tapes_start

        # get lob end index
        while lob_times[lob_end] < end_time and lob_end < max_lob:       # move lob indicies to end time
            lob_end += 1
        
        # get tapes end index
        while tapes[tapes_end][0] < end_time and tapes_end < max_tapes:  # move tapes indicies to end time
            tapes_end += 1

        # feature calculations
        if tapes_start == tapes_end:                                     # if there is no tapes data
            AP = np.nan                                                  # set tapes features to np.nan
            VOL = np.nan
            PSTD = np.nan
            nUoD = np.nan
        else:
            tapes_slice = tapes[tapes_start:tapes_end]                   # extract tapes slice, calculate AP, VOL
            tapes_list = []
            
            for row in tapes_slice:
                for _ in range(int(row[2])):
                    tapes_list.append(row[1])

            tapes_list = np.array(tapes_list, dtype=np.int32)
            AP = np.mean(tapes_list)
            VOL = np.sum(tapes_slice[:,2])
            PSTD = np.std(tapes_list)

            tapes_price_diff = tapes_slice[:,1][1:] - tapes_slice[:,1][:-1]
            n_ups = np.sum(tapes_price_diff > 0)
            n_downs = np.sum(tapes_price_diff < 0)
            nUoD = (n_ups + 1) / (n_downs + 1) - 1

        if lob_start == lob_end:                                         # if there is no LOB data
            MP = np.nan                                                  # set lob features to np.nan
            HIBID = np.nan
            LOASK = np.nan
            SPREAD = np.nan
            TCBS = np.nan
            TCAS = np.nan
            WBP = np.nan
            WAP = np.nan
            AWS = np.nan
            ALPHA = np.nan
            BETA = np.nan
            ZETA = np.nan  
        else:
            lob_slice = lob_data[lob_start:lob_end]                       # extract slices of data 
            LA_HB_a_b_slice = LA_HB_a_b[lob_start:lob_end]                

            # midprice_calcs, alpha, beta
            if median:                                                    # calculate price features
                HIBID = np.median(LA_HB_a_b_slice[:,1])                   # using median if set to true
                LOASK = np.median(LA_HB_a_b_slice[:,0])
                ALPHA = np.median(LA_HB_a_b_slice[:,2])
                BETA = np.median(LA_HB_a_b_slice[:,3])
            else:
                HIBID = np.nanmean(LA_HB_a_b_slice[:,1])
                LOASK = np.nanmean(LA_HB_a_b_slice[:,0])
                ALPHA = np.nanmean(LA_HB_a_b_slice[:,2])
                BETA = np.nanmean(LA_HB_a_b_slice[:,3])

            MP = (HIBID + LOASK) / 2
            SPREAD = LOASK - HIBID
            ZETA = BETA - ALPHA

            if HIBID >= LOASK:
                print("WARNING: HIBID >= LOASK")

            # consolidated calcs
            cas[:] = 0                                                      # reset cas, cbs arrays for new data
            cbs[:] = 0 

            window_index = np.where(window_data[:,0] == end_time)[0]
            if len(window_index) == 1:
                w_bid = window_data[window_index[0], 4]
                w_ask = window_data[window_index[0], 5]
                LOWIN = window_data[window_index[0], 1]
                HIWIN = window_data[window_index[0], 2]
            else:
                w_bid = MP - 100
                w_ask = MP + 100
                LOWIN = 0
                HIWIN = 0

            for ci in prange(int(np.floor(w_bid) - 1), int(np.ceil(w_ask) + 2)):
                # can optimise with LOASK AND HIBID here
                                                                        # only calculate cbs between window left of MP
                cbs_vec = lob_slice[:,ci].copy() * -1                   # and less than LOASK + 100 for efficiency
                cbs_vec[cbs_vec <= 0] = 0                               # idk if this breaks things for efficiency ?:
                cbs[ci] = np.sum(np.abs(np.diff(cbs_vec))) + cbs_vec[0]

                                                                        # only calculate cas between window right of MP
                cas_vec = lob_slice[:,ci].copy()                        # and greater than HIBID - 100 for efficiency
                cas_vec[cas_vec <= 0] = 0                               # idk if this breaks things for efficiency ?:
                cas[ci] = np.sum(np.abs(np.diff(cas_vec))) + cas_vec[0]

            TCBS = np.sum(cbs)                                              # Total CBS
            TCAS = np.sum(cas)                                              # Total CAS

            if TCBS == 0:                                                   # Calculate WBP, np.nan if no activity
                WBP = np.nan
            else:
                WBP = 0
                for ci in prange(800):
                    WBP += (ci + 1) * (cbs[ci] / TCBS)

            if TCAS == 0:                                                   # Calculate WAP, np.nan if no activity
                WAP = np.nan
            else:
                WAP = 0
                for ci in prange(800):
                    WAP += (ci + 1) * (cas[ci] / TCAS)

            AWS = WAP - WBP                                                 # Activity weighted spread calc

        # feature setting
        feat_arr[row_i][features.index("AP")] = AP                          # set the values to the feat_arr
        feat_arr[row_i][features.index("VOL")] = VOL
        feat_arr[row_i][features.index("MP")] = MP
        feat_arr[row_i][features.index("HIBID")] = HIBID
        feat_arr[row_i][features.index("LOASK")] = LOASK
        feat_arr[row_i][features.index("SPREAD")] = SPREAD
        feat_arr[row_i][features.index("TCAS")] = TCAS
        feat_arr[row_i][features.index("TCBS")] = TCBS
        feat_arr[row_i][features.index("WAP")] = WAP
        feat_arr[row_i][features.index("WBP")] = WBP
        feat_arr[row_i][features.index("AWS")] = AWS
        feat_arr[row_i][features.index("ALPHA")] = ALPHA
        feat_arr[row_i][features.index("BETA")] = BETA
        feat_arr[row_i][features.index("ZETA")] = ZETA
        feat_arr[row_i][features.index("GAP")] = MP - AP
        feat_arr[row_i][features.index("ENDT")] = end_time
        feat_arr[row_i][features.index("PSTD")] = PSTD
        feat_arr[row_i][features.index("LOWIN")] = LOWIN
        feat_arr[row_i][features.index("HIWIN")] = HIWIN
        feat_arr[row_i][features.index("nUoD")] = nUoD


        # adjust start times
        start_time = end_time                                                # Set the next start times and 
        lob_start = lob_end                                                  # indicies to the last end times / indicies
        tapes_start = tapes_end

    return feat_arr, features

window_length_s = 5
c = 0
all_features = []
for lob, lob_times, tapes in get_data_gen():
    print(c, end = "\r")
    outside, w_bids, w_asks = get_tapes_window(tapes)
    window_data = np.zeros((len(outside), 6), dtype = float)
    for i in range(len(outside)):
        window_data[i][:4] = outside[i]
        window_data[i][4] = w_bids[i]
        window_data[i][5] = w_asks[i]

    features = get_features(lob, lob_times, tapes, window_length_s, window_data)
    all_features.append(features)
    c += 1

In [63]:
import pandas as pd

dfs = [pd.DataFrame(fa, columns=f) for fa, f in all_features]
for i, df in enumerate(dfs):
    df["WMP"] = (df["WBP"] + df["WAP"]) / 2
    df["MP_diff(y)"] = df["MP"].diff(1)
    
    df["MP_perc(y)"] = df["MP_diff(y)"] / df["MP"]
    df["DAY"] = i
df.describe()

,MP,HIBID,LOASK,AP,WBP,WAP,TCBS,TCAS,AWS,VOL,...,ZETA,ENDT,PSTD,LOWIN,HIWIN,nUoD,WMP,MP_diff(y),MP_perc(y),DAY
count,6119.000000,6119.000000,6120.000000,5858.000000,5882.000000,5981.000000,6120.000000,6120.000000,5750.000000,5858.000000,...,6119.000000,6120.000000,5858.000000,6120.0,6120.000000,5858.000000,5750.000000,6118.000000,6118.000000,6120.0
mean,137.610372,106.622519,168.586853,108.015732,83.529185,153.612509,139.794281,68.480719,67.181767,9.462786,...,1.715460,15302.500000,1.168765,0.0,0.131373,-0.122745,116.603008,0.014577,-0.033096,124.0
std,33.185344,7.757566,64.878119,3.036246,14.291227,37.520198,79.645304,41.110420,29.681157,5.423445,...,1.889333,8834.180777,1.036306,0.0,1.334813,0.656511,20.302044,40.268319,0.258864,0.0
min,49.702381,2.928571,93.048780,93.000000,4.071429,96.262295,0.000000,0.000000,-6.015152,1.000000,...,-2.893889,5.000000,0.000000,0.0,0.000000,-0.888889,63.550000,-190.273417,-1.307611,124.0
25%,114.778424,106.893569,121.230369,107.000000,73.808762,126.875000,85.000000,39.000000,49.862199,5.000000,...,0.462670,7653.750000,0.441601,0.0,0.000000,-0.500000,102.400968,-18.215096,-0.151221,124.0
50%,126.898148,108.434783,145.757979,108.500000,82.892938,143.270588,131.000000,61.000000,63.833507,9.000000,...,1.090147,15302.500000,1.000000,0.0,0.000000,-0.333333,111.660144,-5.452147,-0.046152,124.0
75%,151.572170,109.756837,195.794596,109.900000,93.831280,171.048387,185.000000,93.000000,82.286642,13.000000,...,2.422440,22951.250000,1.685083,0.0,0.000000,0.000000,127.414002,16.717696,0.121167,124.0
max,336.457627,113.389831,566.864407,114.000000,113.000000,422.594595,614.000000,264.000000,368.088345,34.000000,...,15.880735,30600.000000,8.710211,0.0,38.000000,6.000000,238.550422,220.039591,0.663839,124.0


# Extract features from LOB and Tapes

get mean, std, trend, delta from 60 min segments

In [64]:
# code

features = ["TCAS", "TCBS", "ALPHA", "BETA", "ZETA",
            "WMP", "AWS", "VOL", "GAP", "nUoD", "PSTD"]

sum_features = ["LOWIN", "HIWIN"]

y_feats = ["MP_diff(y)", "MP_perc(y)", "DAY"]

X_dfs = []



for c, df in enumerate(dfs):
    print(c, end = "\r")

    X_df = pd.DataFrame()

    for i in range(len(df) - 60):
        train_segment = df[i:i+60]
        test_segment = df[i+60:i+61]

        row = {}
        for f in features:
            # std
            row[f+"_std"] = np.std(train_segment[f])
            # mean
            if f != "WMP": # exclude mean midprice
                row[f+"_mean"] = np.mean(train_segment[f])
                row[f+"_close"] = train_segment[f].iloc[-1]
            # delta
            row[f+"_delta"] = train_segment[f].iloc[-1] - train_segment[f].iloc[0]
            # trend
            row[f+"_corr"] = np.corrcoef(np.arange(len(train_segment[f])), train_segment[f].to_numpy())[0, 1]

        for f in sum_features:
            row[f+"_sum"] = np.sum(train_segment[f])

        for yf in y_feats:
            row[yf] = test_segment[yf].iloc[0]
        row["ENDT"] = test_segment["ENDT"]

        X_df = pd.concat([X_df, pd.DataFrame(row)])

    X_dfs.append(X_df)

X_dfs[0]

,TCAS_std,TCAS_mean,TCAS_close,TCAS_delta,TCAS_corr,TCBS_std,TCBS_mean,TCBS_close,TCBS_delta,TCBS_corr,...,PSTD_mean,PSTD_close,PSTD_delta,PSTD_corr,LOWIN_sum,HIWIN_sum,MP_diff(y),MP_perc(y),DAY,ENDT
60,39.435809,91.316667,80.0,8.0,-0.087793,76.603713,154.933333,236.0,205.0,0.003241,...,1.246211,0.000000,NaN,NaN,0.0,0.0,-3.744949,-0.014051,0,305.0
61,41.205326,93.233333,187.0,26.0,-0.031857,75.004065,157.583333,190.0,120.0,-0.031917,...,1.252418,1.600000,NaN,NaN,0.0,0.0,-3.321256,-0.012618,0,310.0
62,40.433605,91.583333,62.0,-78.0,-0.005201,74.598814,157.983333,94.0,21.0,-0.090751,...,1.305758,4.346135,3.023259,NaN,0.0,0.0,1.460145,0.005517,0,315.0
63,40.572226,89.833333,35.0,-58.0,-0.009749,74.141102,160.383333,217.0,32.0,-0.102357,...,1.303052,1.165922,-0.554543,NaN,0.0,0.0,-0.569652,-0.002157,0,320.0
64,41.262089,88.800000,31.0,-89.0,-0.047814,75.727910,162.016667,283.0,-192.0,-0.044702,...,1.325510,3.023060,1.302979,NaN,0.0,0.0,-1.910348,-0.007286,0,325.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,36.446990,82.183333,26.0,-37.0,0.007855,59.920807,136.283333,226.0,74.0,-0.053973,...,1.614588,0.000000,0.000000,NaN,0.0,0.0,-3.279304,-0.011390,0,30580.0
6116,36.415484,82.250000,67.0,2.0,-0.019438,67.132200,139.966667,373.0,233.0,0.058805,...,1.631128,0.942809,-1.334799,NaN,0.0,0.0,-2.034188,-0.007115,0,30585.0
6117,36.714877,81.866667,42.0,-81.0,-0.064194,67.890375,141.283333,219.0,70.0,0.091213,...,1.697071,6.036348,4.155999,NaN,0.0,0.0,5.287167,0.018158,0,30590.0
6118,36.322395,81.183333,82.0,-35.0,-0.031543,70.107148,143.433333,278.0,83.0,0.146924,...,1.700208,2.059126,2.059126,NaN,0.0,0.0,-7.249827,-0.025534,0,30595.0


In [65]:
merged_dfs = pd.concat(X_dfs[:], axis = 0)
merged_dfs.to_csv(f"final_merged_{window_length_s}.csv")

# Exploratory Model

In [ ]:


clean_merge_dfs = merged_dfs.dropna(how="any")

x_features = list(X_df)
x_features.remove(y_feat)

X = clean_merge_dfs[x_features]
y = 1 / (1 + np.exp(-clean_merge_dfs[y_feat]/4))

baseline_mae = np.mean(np.abs(y - 0.5))
print("Baseline MAE: ", baseline_mae)
print("Samples: ", len(merged_dfs))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming X contains your input features and y contains your target values
# X shape: (number of samples, 35)

# Step 1: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

# Step 2: Normalize the input features
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

DTR = DecisionTreeRegressor(max_leaf_nodes=10)
DTR.fit(X_train, y_train)

ypred = DTR.predict(X_test)
print(mean_absolute_error(ypred, y_test))

plt.figure(figsize=(15,8))
plot_tree(DTR, feature_names=x_features)
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(ypred)
plt.plot(y_test.to_numpy())
plt.xlim(0,1000)

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(x_features, DTR.feature_importances_)
plt.xticks(rotation=90)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

svc = SVR()
svc.fit(X_train, y_train)

ypred = SVR.predict(X_test)
print(mean_absolute_error(ypred, y_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

def create_regression_model(input_shape):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=input_shape),
        layers.Dropout(0.2),  # Add Dropout layer
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation = 'sigmoid')  # Output layer with single neuron for regression
    ])

    opt = optimizers.Adam(learning_rate=0.05)
    model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['mae'])
    return model

input_shape = (len(x_features),)
regression_model = create_regression_model(input_shape)

# Display model architecture
regression_model.summary()

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
history = regression_model.fit(X_train_normalized, y_train, epochs=1000, batch_size=512, validation_split=0.5, callbacks=[early_stopping])

In [ ]:
import matplotlib.pyplot as plt
# Plot training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.axhline(baseline_mae, c = "black", linestyle = "--")
plt.show()

test_loss = regression_model.evaluate(X_test_normalized, y_test)
model_filename = f"regression_model_test_score_{test_loss[-1]:.4f}.keras"
regression_model.save(model_filename)
# Print the test loss
print("Test Loss:", test_loss)

# Save features

for use in models

In [ ]:
merged_dfs.to_csv("final_merged_5s.csv")


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

merged_dfs = pd.read_csv("final_merged_15s.csv", index_col=0)
plt.figure(figsize=(50,40))
sns.heatmap(merged_dfs.corr(), vmin = -1, vmax = 1)

In [ ]:
from sklearn.decomposition import PCA

merged_dfs = pd.read_csv("final_merged_15s.csv", index_col=0)
x_features = list(merged_dfs)
x_features.remove("MP_diff(y)")
x_features.remove("LOWIN_sum")

clean_dfs = merged_dfs.dropna(how="any")

for col in x_features:
    std = clean_dfs[col].std()
    mean = clean_dfs[col].mean()

    clean_dfs[col] = (clean_dfs[col] - mean) / std

pca = PCA(n_components=20)
pca_results = pca.fit_transform(clean_dfs[x_features])

plt.plot(pca.explained_variance_ratio_)

In [ ]:
import numpy as np
plt.figure(figsize=(30,30))
plt.scatter(pca_results[:,0], pca_results[:,1], c = np.log(clean_dfs["MP_diff(y)"]))